<a href="https://colab.research.google.com/github/elaineye117/flight_crash_analysis/blob/master/SI_649_Common_Causes_of_Failures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SI 649 Fun With Python

In [112]:
#sources: https://pythonspot.com/tokenizing-words-and-sentences-with-nltk/,
#python.org/documenation
#https://www.nltk.org/book/
#stackoverflow here and there 

import nltk
import pandas as pd
import numpy as np

file2 = 'https://raw.githubusercontent.com/lhamini/VisQ_Aairplane/master/Airplane_Crashes_and_Fatalities_Since_1908_20190820105639.csv'
safety = pd.read_csv(file2, sep=',', encoding="latin-1")
safety.head() 

#Some simple facts about the dataset #2 
#Total occurances per filed 
airplane_info2 = pd.read_csv(file2)
airplane_info2.head()

data_subset = airplane_info2[['Operator','Summary', 'Date']]
data_subset.head()

#Current data set size
print("Original data set size: ",len(data_subset))
#Drop nan's
no_nans= data_subset.dropna()
print("Size after drop nans: ",len(no_nans))

operator = no_nans['Operator']
summary = no_nans['Summary']
date = no_nans['Date']


#Now we need to loop through each summary clean it from stop-words & punctuation
#Stop words, are just words without any inheriten meaning ( and, so, ...)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import string 

#List of stop words & punctuation symbols 
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words('english')
more_stop_words = ['the', '.',"'s"]
punct = string.punctuation

#summaries without stop words & punctuation & numbers 
#determine most important [words, frequency, operator] for each summary

clean_summaries= []

for x in range(len(summary)):
    example_sent =summary.iloc[x]
    word_tokens = word_tokenize(example_sent) 
    word_tokens2=[x.lower() for x in word_tokens]
    filtered_sentence = [w for w in word_tokens2 if not w in stop and w not in punct and w not in more_stop_words and not w.isdigit() ]
    #remove duplicates before appending
    filtered_sentence_2 = list(set(filtered_sentence))
    token= word_tokenize(operator.iloc[x])
    tha_date = date.iloc[x]
    #determine if military, private, or commercial
    tha_type=""
    if(token[0].lower()== 'military'):
        tha_type = 'military'
        
    elif(token[0].lower()== 'private'):
        tha_type = 'private'
        
    else:
        tha_type = 'commercial'
    
    for y in filtered_sentence_2:
        temp = (y, 1, tha_type,tha_date)
        clean_summaries.append(temp)

#Stemming
#crashing become crash
from nltk.stem import PorterStemmer
clean_summary_2 = []
ps = PorterStemmer()

for word in clean_summaries:
    w= ps.stem(word[0])
    if(w not in stop and w not in punct and w not in more_stop_words and not w.isdigit()):

        temp = (w, word[1], word[2], word[3])
        clean_summary_2.append(temp)
print(date)

Original data set size:  4967
Size after drop nans:  4894
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0       09/17/1908
2       07/12/1912
3       08/06/1913
4       09/09/1913
5       10/17/1913
           ...    
4962    04/16/2019
4963    05/05/2019
4964    05/05/2019
4965    06/03/2019
4966    07/30/2019
Name: Date, Length: 4894, dtype: object


In [113]:
#Top 65 words
top_fifty=[      
'weather','fog','rain','ice','thunderstorm','wind','turbul','cloud','snow','storm','lightn','snowstorm','windshear', 'rainstorm','sandstorm',
 'failur','accid','error','problem','malfunct','crack','leak', 
  "engin",'rotor','rudder', 'wing','brake','skid','tire','fuselag',
  'shot','explod','explos','deton','missil','bomb',
    "mountain",'mountainsid','mount','cliff','ravin',
     'struck','hit','impact','collis',
      'cargo','weight', 'overload',
      'ground','takeoff', 'take-Off',
       'hijack','sabotag' ,'attack',  
       'fire','flame','burn', 
'lost', 
'tree',
'fuel',
'visibl',
'fatigu',
'autopilot',
'volcano',
'overspeed',  
'pilot'     
]

top_causes = []
#Append each of the top 70 words in a new list, for every time it appears in the data set
for x in clean_summary_2:
  if(x[0] in top_fifty):
    top_causes.append(x)
   
#Sort so that it appears uniform colored in the graaph
s= sorted(top_causes, key=lambda i: ( i[0], i[2] ), reverse=False)

#Categories
weather = ['weather','fog','rain','ice','thunderstorm','wind','turbul','cloud','snow','storm','lightn','snowstorm','windshear', 'rainstorm','sandstorm']
failure = ['failur','accid','error','problem','malfunct','crack','leak']
equipement = ["engin",'rotor','rudder', 'wing','brake','skid','tire','fuselag']
explosion = [ 'shot','explod','explos','deton','missil','bomb']
mountain = ["mountain",'mountainsid','mount','cliff','ravin']
collision = [ 'struck','hit','impact','collis']
overload =[     'cargo','weight', 'overload']
take_off = ['ground','takeoff', 'take-Off']
hijack = [ 'hijack','sabotag' ,'attack'] 
fire = ['fire','flame','burn']


categorized = []
for x in s:
  if(x[0]in weather):
    y='weather'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  elif(x[0] in failure):
    y='failure'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  elif(x[0] in equipement):
    y='equipement'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  elif(x[0] in explosion):
    y='explosion'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  elif(x[0] in mountain):
    y='mountain'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  elif(x[0] in collision):
    y='collision'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  elif(x[0] in overload):
    y='overload'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  elif(x[0] in take_off):
    y='takeoff'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  elif(x[0] in hijack):
    y='hijack'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  elif(x[0] in fire):
    y='fire'
    temp = (y,x[1],x[2],x[3])
    categorized.append(temp)

  else:
    categorized.append(x)
s2= sorted(categorized, key=lambda i: ( i[0], i[2] ), reverse=False)

import altair as alt 
#Create dataset & Upload on GitHub
df3= pd.DataFrame(s)
df2 = pd.DataFrame(s2)
df2.to_csv('Top_Words_By_Category_and_Date.csv', index=False)
#This allows to have more than 5000 observation (**performance does suffer)
alt.data_transformers.disable_max_rows()

#Wilkinson Chart
alt.Chart(df2,title = "TOP 70 CAUSES").mark_circle().transform_window(
    id='rank()',
    groupby=['0']
).encode(
    alt.Y('0:O', sort=alt.EncodingSortField(field="id",order='descending'), ),
    alt.X('id:O',title = "Frequency",),
    color=alt.Color('2:O', legend=alt.Legend(title="Legend",orient="left"),scale=alt.Scale(scheme='set1')  ),
    tooltip = ['0:O','2:O']
     
).configure_axis(
    grid=False
).properties(width = 10000)


alt.Chart(...)

#Check out the same chart done in Tableau
-Log Scale

https://public.tableau.com/views/FrequentCrashCauses-LogScale/Sheet1?:display_count=y&:origin=viz_share_link



-Normal Scale

https://public.tableau.com/views/FrequentCrashCauses-NormalScale/Sheet12?:display_count=y&:origin=viz_share_link


-Crashes over the Years

https://public.tableau.com/shared/P5Y6583HC?:display_count=y&:origin=viz_share_link

In [114]:
#Not really  good chart
#TODO: Replicate visualization 1 
line_chart=alt.Chart(df2).mark_line().transform_window(
  frequency='sum(1)', 
  groupby=['0']
).encode(
    alt.X('frequency:Q'),
   alt.Y('0:O',sort=alt.EncodingSortField(field="frequency_word",order='descending') ),
  color=alt.Color('0:O', legend=alt.Legend(title="Legend",orient="left") ),

  tooltip = ['3:T','0:O', ]
)


line_chart2=alt.Chart(df2).mark_line().transform_window(
  frequency='sum(1)', 
  groupby=['0']
).encode(
    alt.Y('frequency:Q'),
   alt.X('3:T',sort=alt.EncodingSortField(field="frequency_word",order='descending') ),
  color=alt.Color('0:O', legend=alt.Legend(title="Legend",orient="left") ),

  tooltip = ['3:T','0:O', ]
)

line_chart+ line_chart2

alt.LayerChart(...)